##Importing required libraries

In [1]:
import pandas as pd
from bs4 import *
import requests
import json
import folium
import geopy
from geopy.geocoders import Nominatim

##Webscrapping data

In [2]:
#Fetching covid vaccination data
path1='https://www.mygov.in/sites/default/files/covid/vaccine/vaccine_counts_today.json'
page1 = requests.get(path1)
soup1 = BeautifulSoup(page1.content, 'html.parser')
site_json=json.loads(soup1.text)
site_json['vacc_st_data'][0]
vaccine_data=pd.DataFrame(columns=['state','dose1','dose2','total_doses'])
c=0
for i in site_json['vacc_st_data']:
  vaccine_data.loc[c]=[i['covid_state_name'],i['dose1'],i['dose2'],i['total_doses']]
  c+=1
#Fetching covid cases data
path2='https://www.mygov.in/sites/default/files/covid/covid_state_counts_ver1.json'
page2 = requests.get(path2)
soup2 = BeautifulSoup(page2.content, 'html.parser')
site_json2=json.loads(soup2.text)
active_cases=site_json2['Active']
state_names=site_json2['Name of State / UT']
total_deaths=site_json2['Death']
cured_cases=site_json2['Cured/Discharged/Migrated']
total_cases=site_json2['Total Confirmed cases']
covid_data=pd.DataFrame(columns =['state', 'total_cases','active_cases','discharged_cases','total_deaths'])
covid_data.loc[0]=[state_names['0'],total_cases['0'],active_cases['0'],cured_cases['0'],total_deaths['0']]
for i in range(0,36):
  covid_data.loc[i]=[state_names[str(i)],total_cases[str(i)],active_cases[str(i)],cured_cases[str(i)],total_deaths[str(i)]]

##Data Exploration

In [3]:
covid_data

,state,total_cases,active_cases,discharged_cases,total_deaths
0,Andaman and Nicobar,7648,8,7511,129
1,Andhra Pradesh,2063177,5222,2043616,14339
2,Arunachal Pradesh,55089,142,54667,280
3,Assam,608450,3968,598515,5967
4,Bihar,726054,36,716357,9661
5,Chandigarh,65321,27,64474,820
6,Chhattisgarh,1005827,230,992025,13572
7,Dadra and Nagar Haveli and Daman and Diu,10679,4,10671,4
8,Delhi,1439566,334,1414141,25091
9,Goa,177852,574,173920,3358


In [4]:
vaccine_data

,state,dose1,dose2,total_doses
0,Andaman and Nicobar,292988,186655,479643
1,Andhra Pradesh,31444989,18032266,49477255
2,Arunachal Pradesh,768011,507279,1275290
3,Assam,19853518,7072829,26926347
4,Bihar,48394063,15912408,64306471
5,Chandigarh,921706,521520,1443226
6,Chhattisgarh,14415425,6646148,21061573
7,Dadra and Nagar Haveli and Daman and Diu,657198,352146,1009344
8,Delhi,12867945,7104856,19972801
9,Goa,1252224,851685,2103909


In [5]:
covid_data[['dose1','dose2','total_doses']] = vaccine_data[['dose1','dose2','total_doses']]
covid_data.head()

,state,total_cases,active_cases,discharged_cases,total_deaths,dose1,dose2,total_doses
0,Andaman and Nicobar,7648,8,7511,129,292988,186655,479643
1,Andhra Pradesh,2063177,5222,2043616,14339,31444989,18032266,49477255
2,Arunachal Pradesh,55089,142,54667,280,768011,507279,1275290
3,Assam,608450,3968,598515,5967,19853518,7072829,26926347
4,Bihar,726054,36,716357,9661,48394063,15912408,64306471


##Date Visualization

In [6]:
india=folium.Map(location = [20.5937,78.9629],zoom_start=4.5)

In [7]:
geolocator = Nominatim(timeout=10, user_agent = "myGeolocator")
covid_data['gcode'] = covid_data.state.apply(geolocator.geocode)
covid_data['lat'] = [g.latitude for g in covid_data.gcode]
covid_data['long'] = [g.longitude for g in covid_data.gcode]
covid_data.head()


,state,total_cases,active_cases,discharged_cases,total_deaths,dose1,dose2,total_doses,gcode,lat,long
0,Andaman and Nicobar,7648,8,7511,129,292988,186655,479643,"(Andaman, India, (10.0001051, 93.0000194))",10.000105,93.000019
1,Andhra Pradesh,2063177,5222,2043616,14339,31444989,18032266,49477255,"(Andhra Pradesh, India, (15.9240905, 80.1863809))",15.924091,80.186381
2,Arunachal Pradesh,55089,142,54667,280,768011,507279,1275290,"(Arunachal Pradesh, India, (28.0937702, 94.592...",28.093770,94.592133
3,Assam,608450,3968,598515,5967,19853518,7072829,26926347,"(Assam, India, (26.4073841, 93.2551303))",26.407384,93.255130
4,Bihar,726054,36,716357,9661,48394063,15912408,64306471,"(Bihar, India, (25.6440845, 85.906508))",25.644085,85.906508


In [8]:
for state,lat,long,total_cases,Death,Recov,Active,dose1,dose2,totaldoses in zip(list(covid_data['state']),list(covid_data['lat']),list(covid_data['long']),
                                                         list(covid_data['total_cases']),list(covid_data['total_deaths']),list(covid_data['discharged_cases']),
                                                         list(covid_data['active_cases']),list(covid_data['dose1']),list(covid_data['dose2']),
                                                         list(covid_data['total_doses'])):
    #for creating circle marker
    folium.CircleMarker(location = [lat,long],
                       radius = 5,
                       color='red',
                       fill = True,
                       fill_color="red").add_to(india)
    #for creating marker
    folium.Marker(location = [lat,long],
                  # adding information that need to be displayed on popup
                  popup=folium.Popup(('<strong><b>State  : '+state+'</strong> <br>' +
                    '<strong><b>Total Cases : '+total_cases+'</strong><br>' +
                    '<strong><font color= red>Deaths : </font>'+Death+'</strong><br>' +
                    '<strong><font color=green>Recoveries : </font>'+Recov+'</strong><br>' +
                    '<strong><b>Active Cases : '+Active+'</strong><br>'+
                    '<strong><b><font color=green>Vaccine data : </font></strong> <br>' +
                    '<strong><b>Dose 1 : '+dose1+'</strong><br>' +
                    '<strong><b>Dose 2  : '+dose2+'</strong><br>' +
                    '<strong><b>Total Doses : '+totaldoses+'</strong><br>'),max_width=200)).add_to(india)

In [9]:
india